In [12]:
#2018 HMDA Edit Testing File Generator
from collections import OrderedDict
import json
import os
import pandas as pd
import random

#custom imports
import lar_constraints
import lar_generator
from rules_engine import rules_engine


In [2]:
#2018 Filing Instruction Guide: https://www.consumerfinance.gov/data-research/hmda/static/for-filers/2018/2018-HMDA-FIG.pdf

use_cols = ['name', 'metDivName', 'countyFips', 'geoIdMsa', 'metDivFp', 'smallCounty', 'tracts']
cbsa_cols = ['name', 'metDivName', 'state', 'countyFips', 'county', 'tracts','geoIdMsa', 'metDivFp', 'smallCounty', 
             'stateCode', 'tractDecimal']
cbsas = pd.read_csv('../dependancies/tract_to_cbsa_2015.txt', usecols=use_cols, delimiter='|', 
                    header=None, names=cbsa_cols, dtype=str) #load tract to CBSA data from platform file
cbsas["tractFips"] = cbsas.countyFips + cbsas.tracts
counties = list(cbsas.countyFips)
tracts = list(cbsas.tractFips)



In [3]:
#load schemas for LAR and transmittal sheet
lar_schema_df = pd.DataFrame(json.load(open("../schemas/lar_schema.json", "r")))
ts_schema_df = pd.DataFrame(json.load(open("../schemas/ts_schema.json", "r")))


In [4]:
lar_gen = lar_generator.lar_gen(lar_schema_df, ts_schema_df, counties=counties, tracts=tracts) #instantiate generator
lar_const = lar_constraints.lar_constraints(counties=counties, tracts=tracts)#instantiate constraints

validator = rules_engine(lar_schema_df, ts_schema_df)

In [6]:


#test = lar_generator.lar_gen(lar_schema_df, ts_schema_df)
row = lar_gen.make_row() #create test row

#run constraint functions on row

file_length = 100 #set number of rows in test file

first = True
for i in range(file_length):
    print("creating row number: {num}".format(num=i))
    row = lar_gen.make_row() #create new row
    diff = [1] #initialize diff for loop
    while len(diff) >0:
        row_base = row.copy() #copy row to enable diff
        #apply constraint functions to LAR row
        for func in lar_const.constraint_funcs: 
            row = getattr(lar_const, func)(row)
            #convert initial and copied rows to sets for diff
            initial_row = set(row_base.items())
            changed_row = set(row.items())
            diff = (initial_row - changed_row)
#            print(len(initial_row), len(changed_row)) #check number of fields, should be 110
            #show readout of number of changes made to new LAR row
            #print("changes from {func}: {vals}".format(func=str(func), vals=len(diff)))
    
    #create first row of dataframe
    if first:
        lar_frame = pd.DataFrame(row, index=[1])
        first = False
    #add additional rows to dataframe
    else:
        print("concating")
        new_lar = pd.DataFrame(row, index=[1])
        lar_frame = pd.concat([lar_frame, new_lar], axis=0)




creating row number: 0
changes from v612_const: 1
changes from v610_const: 1
changes from v613_const: 2
changes from v614_const: 3
changes from v615_const: 4
changes from v619_const: 4
changes from v622_const: 4
changes from v627_const: 4
changes from v628_const: 6
changes from v629_const: 6
changes from v630_const: 7
changes from v631_const: 9
changes from v632_const: 10
changes from v633_const: 10
changes from v634_const: 10
changes from v635_const: 11
changes from v636_const: 11
changes from v637_const: 12
changes from v638_const: 12
changes from v638_const: 12
changes from v640_const: 12
changes from v641_const: 13
changes from v643_const: 13
changes from v644_const: 13
changes from v645_const: 13
changes from v647_const: 13
changes from v648_const: 14
changes from v649_const: 14
changes from v650_const: 14
changes from v651_const: 14
changes from v652_const: 14
changes from v654_const: 15
changes from v655_const: 15
changes from v656_const: 16
changes from v657_const: 16
changes f

changes from v667_const: 2
changes from v668_const: 2
changes from v669_const: 2
changes from v670_const: 1
changes from v671_const: 1
changes from v672_const: 1
changes from v673_const: 1
changes from v674_const: 1
changes from v675_const: 1
changes from v676_const: 1
changes from v677_const: 1
changes from v678_const: 1
changes from v679_const: 1
changes from v680_const: 1
changes from v681_const: 1
changes from v682_const: 1
changes from v688_const: 1
changes from v689_const: 1
changes from v690_const: 1
changes from v692_const: 1
changes from v693_const: 1
changes from v694_const: 1
changes from v696_const: 1
changes from v697_const: 1
changes from v698_const: 1
changes from v699_const: 1
changes from v700_const: 1
changes from v701_const: 1
changes from v702_const: 1
changes from v703_const: 1
changes from v704_const: 1
changes from v705_const: 1
changes from v612_const: 0
changes from v610_const: 0
changes from v613_const: 1
changes from v614_const: 1
changes from v615_const: 1
c

changes from v627_const: 1
changes from v628_const: 5
changes from v629_const: 1
changes from v630_const: 1
changes from v631_const: 4
changes from v632_const: 1
changes from v633_const: 1
changes from v634_const: 1
changes from v635_const: 1
changes from v636_const: 1
changes from v637_const: 1
changes from v638_const: 1
changes from v638_const: 1
changes from v640_const: 1
changes from v641_const: 1
changes from v643_const: 1
changes from v644_const: 1
changes from v645_const: 1
changes from v647_const: 1
changes from v648_const: 1
changes from v649_const: 1
changes from v650_const: 1
changes from v651_const: 1
changes from v652_const: 1
changes from v654_const: 1
changes from v655_const: 1
changes from v656_const: 1
changes from v657_const: 1
changes from v658_const: 1
changes from v661_const: 1
changes from v662_const: 1
changes from v663_const: 1
changes from v664_const: 1
changes from v666_const: 1
changes from v667_const: 1
changes from v668_const: 1
changes from v669_const: 1
c

changes from v682_const: 5
changes from v688_const: 5
changes from v689_const: 5
changes from v690_const: 5
changes from v692_const: 5
changes from v693_const: 5
changes from v694_const: 5
changes from v696_const: 5
changes from v697_const: 5
changes from v698_const: 5
changes from v699_const: 5
changes from v700_const: 5
changes from v701_const: 5
changes from v702_const: 5
changes from v703_const: 5
changes from v704_const: 5
changes from v705_const: 5
changes from v612_const: 0
changes from v610_const: 0
changes from v613_const: 1
changes from v614_const: 1
changes from v615_const: 1
changes from v619_const: 1
changes from v622_const: 1
changes from v627_const: 1
changes from v628_const: 1
changes from v629_const: 1
changes from v630_const: 1
changes from v631_const: 4
changes from v632_const: 1
changes from v633_const: 1
changes from v634_const: 1
changes from v635_const: 2
changes from v636_const: 2
changes from v637_const: 2
changes from v638_const: 2
changes from v638_const: 2
c

changes from v699_const: 0
changes from v700_const: 0
changes from v701_const: 0
changes from v702_const: 0
changes from v703_const: 0
changes from v704_const: 0
changes from v705_const: 0
concating
creating row number: 12
changes from v612_const: 0
changes from v610_const: 0
changes from v613_const: 1
changes from v614_const: 1
changes from v615_const: 2
changes from v619_const: 3
changes from v622_const: 3
changes from v627_const: 3
changes from v628_const: 7
changes from v629_const: 5
changes from v630_const: 5
changes from v631_const: 9
changes from v632_const: 7
changes from v633_const: 7
changes from v634_const: 7
changes from v635_const: 9
changes from v636_const: 9
changes from v637_const: 10
changes from v638_const: 13
changes from v638_const: 13
changes from v640_const: 14
changes from v641_const: 14
changes from v643_const: 15
changes from v644_const: 15
changes from v645_const: 15
changes from v647_const: 16
changes from v648_const: 16
changes from v649_const: 16
changes fr

...

changes from v662_const: 2
changes from v663_const: 2
changes from v664_const: 2
changes from v666_const: 2
changes from v667_const: 2
changes from v668_const: 2
changes from v669_const: 2
changes from v670_const: 1
changes from v671_const: 1
changes from v672_const: 1
changes from v673_const: 1
changes from v674_const: 1
changes from v675_const: 1
changes from v676_const: 1
changes from v677_const: 1
changes from v678_const: 1
changes from v679_const: 1
changes from v680_const: 1
changes from v681_const: 1
changes from v682_const: 1
changes from v688_const: 1
changes from v689_const: 1
changes from v690_const: 1
changes from v692_const: 1
changes from v693_const: 1
changes from v694_const: 1
changes from v696_const: 1
changes from v697_const: 1
changes from v698_const: 1
changes from v699_const: 1
changes from v700_const: 1
changes from v701_const: 1
changes from v702_const: 1
changes from v703_const: 1
changes from v704_const: 1
changes from v705_const: 1
changes from v612_const: 0
c

changes from v698_const: 18
changes from v699_const: 18
changes from v700_const: 18
changes from v701_const: 18
changes from v702_const: 18
changes from v703_const: 18
changes from v704_const: 18
changes from v705_const: 18
changes from v612_const: 0
changes from v610_const: 0
changes from v613_const: 1
changes from v614_const: 1
changes from v615_const: 1
changes from v619_const: 1
changes from v622_const: 1
changes from v627_const: 1
changes from v628_const: 1
changes from v629_const: 1
changes from v630_const: 1
changes from v631_const: 1
changes from v632_const: 1
changes from v633_const: 1
changes from v634_const: 1
changes from v635_const: 1
changes from v636_const: 1
changes from v637_const: 1
changes from v638_const: 1
changes from v638_const: 1
changes from v640_const: 1
changes from v641_const: 1
changes from v643_const: 1
changes from v644_const: 1
changes from v645_const: 1
changes from v647_const: 1
changes from v648_const: 1
changes from v649_const: 1
changes from v650_co

changes from v697_const: 14
changes from v698_const: 14
changes from v699_const: 14
changes from v700_const: 14
changes from v701_const: 14
changes from v702_const: 14
changes from v703_const: 14
changes from v704_const: 14
changes from v705_const: 14
changes from v612_const: 0
changes from v610_const: 0
changes from v613_const: 1
changes from v614_const: 1
changes from v615_const: 1
changes from v619_const: 1
changes from v622_const: 1
changes from v627_const: 1
changes from v628_const: 1
changes from v629_const: 1
changes from v630_const: 1
changes from v631_const: 1
changes from v632_const: 1
changes from v633_const: 1
changes from v634_const: 1
changes from v635_const: 1
changes from v636_const: 1
changes from v637_const: 1
changes from v638_const: 1
changes from v638_const: 1
changes from v640_const: 1
changes from v641_const: 1
changes from v643_const: 1
changes from v644_const: 1
changes from v645_const: 1
changes from v647_const: 1
changes from v648_const: 1
changes from v649_c

changes from v669_const: 10
changes from v670_const: 11
changes from v671_const: 11
changes from v672_const: 11
changes from v673_const: 11
changes from v674_const: 11
changes from v675_const: 11
changes from v676_const: 11
changes from v677_const: 11
changes from v678_const: 11
changes from v679_const: 11
changes from v680_const: 11
changes from v681_const: 11
changes from v682_const: 11
changes from v688_const: 11
changes from v689_const: 11
changes from v690_const: 11
changes from v692_const: 11
changes from v693_const: 10
changes from v694_const: 10
changes from v696_const: 10
changes from v697_const: 10
changes from v698_const: 10
changes from v699_const: 10
changes from v700_const: 10
changes from v701_const: 10
changes from v702_const: 10
changes from v703_const: 10
changes from v704_const: 10
changes from v705_const: 10
changes from v612_const: 0
changes from v610_const: 0
changes from v613_const: 1
changes from v614_const: 1
changes from v615_const: 1
changes from v619_const: 

changes from v615_const: 1
changes from v619_const: 2
changes from v622_const: 2
changes from v627_const: 2
changes from v628_const: 6
changes from v629_const: 6
changes from v630_const: 7
changes from v631_const: 11
changes from v632_const: 11
changes from v633_const: 11
changes from v634_const: 12
changes from v635_const: 15
changes from v636_const: 16
changes from v637_const: 16
changes from v638_const: 18
changes from v638_const: 18
changes from v640_const: 18
changes from v641_const: 19
changes from v643_const: 19
changes from v644_const: 19
changes from v645_const: 19
changes from v647_const: 19
changes from v648_const: 19
changes from v649_const: 20
changes from v650_const: 20
changes from v651_const: 20
changes from v652_const: 20
changes from v654_const: 21
changes from v655_const: 21
changes from v656_const: 22
changes from v657_const: 23
changes from v658_const: 23
changes from v661_const: 23
changes from v662_const: 24
changes from v663_const: 24
changes from v664_const: 24

In [8]:
#Quality and Macro field interrelationship constraints:

In [9]:
#Create a sample TS row
#Note: this will need to be more robust to include other federal agencies
# mlo_id needs NA option
#set dummy values for TS row
ts_row_small = OrderedDict()
ts_row_small["record_id"]="1"
ts_row_small["inst_name"]="Ficus Bank"
ts_row_small["calendar_year"]=str(2018)
ts_row_small["calendar_quarter"]="4"
ts_row_small["contact_name"]="Mr. Smug Pockets"
ts_row_small["contact_tel"]="555-555-5555"
ts_row_small["contact_email"]="pockets@ficus.com"
ts_row_small["contact_street_address"]="1234 Ficus Lane"
ts_row_small["office_city"]="Ficusville"
ts_row_small["office_state"]="UT"
ts_row_small["office_zip"]="84096"
ts_row_small["federal_agency"]="9"
ts_row_small["lar_entries"]="100"
ts_row_small["tax_id"]="01-0123456"
ts_row_small["lei"]="12345678901234567890"



In [29]:
#join LAR and TS rows to make an output file

def write_file(ts_input=None, lar_input=None, directory="../edits_files/", name="passes_all.txt"):
    """Takes a TS row as a dictionary and LAR data as a dataframe. Writes LAR data to file and 
    re-reads it to combine with TS data to make a full file."""
    #make directories for files if they do not exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    #write LAR dataframe to file
    parts_dir = directory+"file_parts/"
    if not os.path.exists(parts_dir):
        os.makedirs(parts_dir)
    lar_input.to_csv(parts_dir + "lar_data.txt", sep="|", header=False, index=False)
    #load LAR data as file rows
    with open(parts_dir + "lar_data.txt", 'r') as lar_data:
        lar = lar_data.readlines()
    with open(directory + name, 'w') as final_file:
        final_file.write("|".join(ts_input.values()))
        for line in lar:
            final_file.write("{line}\n".format(line=line))
            
#load LAR and TS data as strings of file rows
#write file rows to same file

In [30]:
write_file(ts_input=ts_row_small, lar_input=lar_frame)